In [1]:
import pandas as pd
import numpy as np
import openpyxl
from warnings import filterwarnings
filterwarnings('ignore')
import researchpy as rp

### Kolonların Adlandırılması & Veri Seti İmport

In [2]:
coll = ['Grubun_veritabani', 'Tarih', 'Yarisma', 'Saha', 'Şema', 'Rakip', 'xG',
       'Sutlar', 'Sutlar_hedef', 'Sut_yuzdesi', 'Paslar', 'Paslar_dogru',
       'Pas_yuzdesi', 'Top_hakimiyeti_yuzdesi', 'Kayiplar', 'Kayiplar_Dusuk',
       'Kayiplar_Orta', 'Kayiplar_Yuksek', 'Kurtarislar', 'Kurtarislar_Dusuk',
       'Kurtarislar_Orta', 'Kurtarislar_Yuksek', 'Cekismeler',
       'Cekismeler_kazanilan', 'Cekismeler_yuzdesi', 'Yenilen_Gol',
       'Atilan_Gol']
df=pd.read_csv("DF.csv", names=coll)
df=df.rename(index={j: i for i, j in enumerate(df.index)})
df['Tarih'] = df['Tarih'].apply(lambda x: pd.to_datetime(x))
df=df.drop_duplicates(keep='last')
df=df.rename(index={j: i for i, j in enumerate(df.index)})
# df.to_excel("df1.xlsx",index=False)

In [3]:
#df

In [4]:
rp.summary_cont(df.loc[df.Grubun_veritabani=="Arsenal",'Sutlar'])
# Arsenal'in maçı var tabloda
# Arsenal'in Şut ortalaması 12.7656
# Arsenal'in Şut ortalaması standart sapması 5.2727
# Arsenal'in Şut ortalaması standart hatası 0.6591
# Arsenal'in Şut ortalaması güven aralıkları 11.4485 - 14.0827

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,Sutlar,64.0,12.7656,5.2727,0.6591,11.4485,14.0827


# MERKEZİ EĞİLİM ÖLÇÜLERİ (Portföy Temsil İstatistiği)

In [14]:
print('Ortalama', df.loc[df.Grubun_veritabani=="Arsenal",'Sutlar'].mean())
print('Median', df.loc[df.Grubun_veritabani=="Arsenal",'Sutlar'].median()) # MEDİAN VE ORTALAMADA ÇOK FARKLILIK YOK. DAĞILIM SİMETRİK

Ortalama 12.765625
Median 13.0


In [6]:
def Merkezi_Egilim(df, clmn, tkm, trgt):
    ort = df.loc[df[clmn]==tkm,trgt].mean()
    med = df.loc[df[clmn]==tkm,trgt].median()
    minn = df.loc[df[clmn]==tkm,trgt].min()
    maxx = df.loc[df[clmn]==tkm,trgt].max()
    basiklik = df.loc[df[clmn]==tkm,trgt].kurtosis()
    frk = ort - med
    summary = rp.summary_cont(df.loc[df[clmn]==tkm,trgt])
    summary_95 = summary['95% Conf.'].values
    summary_Interval = summary['Interval'].values
    StnSapm = summary['SD'].values
    Mac_Sayisi = summary['N'].values
    
    
    return Mac_Sayisi, ort, StnSapm, med, frk, minn, maxx, basiklik, summary_95, summary_Interval 

In [7]:
trgt = 'Atilan_Gol' # MERKEZİ DAĞILIM İÇİN ÖLÇÜLECEK DEĞERLER

setTakimlar=list(set(df['Grubun_veritabani']))
Sontable = pd.DataFrame()
for tkms in setTakimlar:
    table = Merkezi_Egilim( df=df, clmn='Grubun_veritabani', tkm=tkms, trgt=trgt)
    dct = {tkms:[table[0], table[1], table[2], table[3], table[4],  table[5],  table[6], table[7], table[8], table[9]]}
    dfTakimlar = pd.DataFrame(dct, index=[trgt+'_Mac_Sayisi', trgt+'_Ortalama', trgt+'_Standart Sapma', trgt+'_Median', trgt+'_Ort_Med_Fark', trgt+'_Min', trgt+'_Max', trgt+'_Basiklik', trgt+'_95% Conf.', trgt+'_Interval']).T
    Sontable =pd.concat([Sontable, dfTakimlar],axis=0)


# DAĞILIM ÖLÇÜLERİ

### Standart Sapma: Ortalamadan olan sapmaların genel öçlüsü. (Ortalamaya olan uzaklıkların ortalaması)

### Varyans: Standart Sapmaların karesidir. (Ortalamadan olan sapmaların karelerinin ortalaması) - Ortalama etrafındaki dağılımın ölçüsü

### Çarpıklık: Değişken dağılımının simetrik olamayışı. (Dağılımı simetrik olmayan değişkenlerde Medyan değeri ile Merkezi Eğilim temsil edilir.)

#### Çarpıklığın tabloda gözlenmesi (tam hesaplanmadı, sadece ortalama ve medyan farkı)

In [15]:
Sontable[Sontable[trgt+'_Ort_Med_Fark']==0][:10] # sıfırdan küçükse çarpıklık sola - büyükse çarpıklık sağa - 0 ise simetrik (Medyan ve Ortalam Farklar)

,Atilan_Gol_Mac_Sayisi,Atilan_Gol_Ortalama,Atilan_Gol_Standart Sapma,Atilan_Gol_Median,Atilan_Gol_Ort_Med_Fark,Atilan_Gol_Min,Atilan_Gol_Max,Atilan_Gol_Basiklik,Atilan_Gol_95% Conf.,Atilan_Gol_Interval
Crewe Alexandra,[1.0],0.0,[nan],0.0,0.0,0,0,NaN,[nan],[nan]
Athletic Bilbao,[2.0],2.0,[1.4142],2.0,0.0,1,3,NaN,[-10.7062],[14.7062]
Milan,[3.0],1.0,[1.0],1.0,0.0,0,2,NaN,[-1.4841],[3.4841]
Pumas UNAM,[1.0],0.0,[nan],0.0,0.0,0,0,NaN,[nan],[nan]
Rangers,[2.0],1.0,[1.4142],1.0,0.0,0,2,NaN,[-11.7062],[13.7062]
Levante,[1.0],0.0,[nan],0.0,0.0,0,0,NaN,[nan],[nan]
Ludogorets,[2.0],0.5,[0.7071],0.5,0.0,0,1,NaN,[-5.8531],[6.8531]
Las Palmas,[1.0],3.0,[nan],3.0,0.0,3,3,NaN,[nan],[nan]
Celta de Vigo,[1.0],1.0,[nan],1.0,0.0,1,1,NaN,[nan],[nan]
Sevilla,[2.0],0.0,[0.0],0.0,0.0,0,0,NaN,[nan],[nan]


### Basıklık: Değişken Dağılımının sivriliğini / basıklığını gösteren istatistik. (Değişken dağılımının ortalamada merkezlenmiş mi yoksa aşırı uzaklaşmış mı) 
#### BK=3 ise dağılım standart dağılıma uygun.
#### BK>3 ise dağılım sivridir.
#### BK<3 ise dağılım basıktır.

### Basıklığın tabloda gözlenmesi (ölçüt değeri=3)

In [9]:
Sontable[Sontable[trgt+'_Basiklik']<3][20:30] # 3'ten küçükse basık, 3'ten büyükse sivri, 3'e eşitse normaldir.

,Atilan_Gol_Mac_Sayisi,Atilan_Gol_Ortalama,Atilan_Gol_Standart Sapma,Atilan_Gol_Median,Atilan_Gol_Ort_Med_Fark,Atilan_Gol_Min,Atilan_Gol_Max,Atilan_Gol_Basiklik,Atilan_Gol_95% Conf.,Atilan_Gol_Interval
Cardiff City,[7.0],0.857143,[0.6901],1.0,-0.142857,0,2,0.336,[0.2189],[1.4953]
Newcastle United,[44.0],1.090909,[1.0302],1.0,0.090909,0,4,0.09152,[0.7777],[1.4041]
Crystal Palace,[44.0],1.068182,[1.2085],1.0,0.068182,0,5,1.594993,[0.7008],[1.4356]
Tottenham Hotspur,[66.0],1.909091,[1.3893],2.0,-0.090909,0,5,-0.755968,[1.5676],[2.2506]
Everton,[51.0],1.254902,[1.0926],1.0,0.254902,0,5,1.272352,[0.9476],[1.5622]
Leeds United,[45.0],1.4,[1.3551],1.0,0.4,0,5,0.462749,[0.9929],[1.8071]
Sheffield United,[32.0],0.5625,[0.669],0.0,0.5625,0,2,-0.391459,[0.3213],[0.8037]
Porto,[5.0],0.6,[0.5477],1.0,-0.4,0,1,-3.333333,[-0.0801],[1.2801]
Sheffield Wednesday,[6.0],0.5,[0.5477],0.5,0.0,0,1,-3.333333,[-0.0748],[1.0748]
Barnsley,[9.0],0.666667,[0.866],0.0,0.666667,0,2,-1.079365,[0.001],[1.3324]


### Korelasyon: İki değişken arasındaki ilişkiyi, ilişkinin anlamlı olup olmadığını, ilişkinin şiddetini ve yönünü ifade eden istatistiksel teknik.

In [10]:
def Kor_Kov(df, iliski, hedef):
    cor = df[[iliski, hedef]].corr()
    cor=cor.drop([iliski],axis=1)
    
    cov = df[[iliski, hedef]].cov()
    cov=cov.drop([iliski],axis=1)
    
    return cor, cov
    

In [11]:
SonTableCor = pd.DataFrame()
SonTableCov = pd.DataFrame()
for i in df.columns[6:-1]:
    table = Kor_Kov(df=df, iliski=i, hedef=trgt)
    SonTableCor =pd.concat([SonTableCor, table[0]],axis=1)
    SonTableCov =pd.concat([SonTableCov, table[1]],axis=1)
    

In [12]:
SonTableCor.drop(SonTableCor[1:2],axis=0) #hedef değişkenin diğer değişkenlerle korelasyonu

,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol
xG,0.58157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sutlar,NaN,0.40435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sutlar_hedef,NaN,NaN,0.661759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sut_yuzdesi,NaN,NaN,NaN,0.429187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Paslar,NaN,NaN,NaN,NaN,0.262903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Paslar_dogru,NaN,NaN,NaN,NaN,NaN,0.274806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pas_yuzdesi,NaN,NaN,NaN,NaN,NaN,NaN,0.235449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Top_hakimiyeti_yuzdesi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.257258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Kayiplar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.158504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Kayiplar_Dusuk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.117916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Kovaryans: İki değişken arasındaki ilişkinin değişkenlik ölçüsü. (İki rasgele değişkenin kendi ortalamalarından olan sapmalarının çarpımlarının beklenen değeri) - 2 değişkenin birlikte ortaya çıkardığı değişimin incelenmesi

In [13]:
SonTableCov.drop(SonTableCor[1:2],axis=0) #hedef değişkenin diğer değişkenlerle koveryansı

,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol,Atilan_Gol
xG,0.674431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sutlar,NaN,2.704249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sutlar_hedef,NaN,NaN,2.156206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sut_yuzdesi,NaN,NaN,NaN,9.784159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Paslar,NaN,NaN,NaN,NaN,46.657433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Paslar_dogru,NaN,NaN,NaN,NaN,NaN,47.630165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pas_yuzdesi,NaN,NaN,NaN,NaN,NaN,NaN,1.713775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Top_hakimiyeti_yuzdesi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.841704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Kayiplar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.46269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Kayiplar_Dusuk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.262887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
